In [27]:
import time
import qrcode
import cv2
import psycopg2 as psy

class event:
    def __init__(self, data):
        self.name = data.split('-')[1]
        self.org = data.split('-')[0]
    def print_id(self):
        print('ID:', self.name, '-', 'ORG:', self.org)
    def db_connect(self):    
        with open('../password') as pwd:
            self.pwd = pwd.readlines()[0].replace('\n', '')
        self.conn = psy.connect(dbname = 'coffee', user = 'coffeepi', password = self.pwd)
        self.curs = self.conn.cursor()
        print('Connection to database "coffee" established')
    def db_add_coffee(self):
        print('One coffee added to', self.name, '- Total count:', 50)

In [2]:
detector = cv2.QRCodeDetector()
img = cv2.imread('/home/squirry/Pictures/coffee/emilio.png', cv2.IMREAD_GRAYSCALE)
data, bbox, _ = detector.detectAndDecode(img)

In [37]:
ev = event(data)

AttributeError: 'event' object has no attribute 'pwd'

In [23]:
ev.db_connect()
ev.curs.execute("INSERT INTO log (event, username) VALUES (now(), '{}');".format(ev.name))
ev.conn.commit()
ev.conn.close()

In [57]:
import pandas as pd
import psycopg2

with open('../password') as pwd:
    pwd = pwd.readlines()[0].replace('\n', '')

conn = psy.connect(dbname = 'coffee', user = 'coffeepi', password = pwd)
curs = conn.cursor()
curs.execute('''
    SELECT *, balance - due AS total FROM (
        SELECT username, count(*)*.4 AS due, balance FROM log 
            LEFT JOIN balance USING (username)
        GROUP BY log.username, balance
    ) AS foo;
''')

total = curs.fetchall()
pd.DataFrame({'name': [total[i][0] for i in range(len(total))],
              'balance': [-total[i][1] if total[i][3] is None else total[i][3] for i in range(len(total))]})

,name,balance
0,eb97ziwi,9.2
1,Asia Argento,-0.4
